<!-- SOLUTION CELL -->
<h2 style="color: red">This is the solutions file!</h2>

<span style="color: red">We strongly recommend not looking at this file, which contains the solutions to all the exercises, until after the practical is over. Use [`practical.ipynb`](practical.ipynb) instead.</span>

This is the second part of the practical components of the tutorial, on two-sample testing.

It doesn't overlap too much with [the ridge regression section](ridge.ipynb), but we're going to assume you've done (at least the first parts of) that one first.